In [ ]:
print("general libs to download. Use ChatGPT and pip install to handle issues")

#https://huggingface.co/hkunlp/instructor-large
!pip install -q sentence_transformers InstructorEmbedding qdrant_client

import re
import json

from bs4 import BeautifulSoup

import json
from InstructorEmbedding import INSTRUCTOR

import torch
from transformers import BartForConditionalGeneration, BartTokenizer




general libs to download. Use ChatGPT and pip install to handle issues


/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [ ]:
print("Embedding test")

from InstructorEmbedding import INSTRUCTOR
model = INSTRUCTOR('hkunlp/instructor-large')

sentence = "Hello AI world"
instruction = "hello"

embeddings = model.encode([[instruction,sentence]])
print(embeddings[0])


Embedding test
load INSTRUCTOR_Transformer
max_seq_length  512
[-5.67759424e-02 -7.85743538e-03 -1.55622950e-02  2.27734400e-03
  3.52524444e-02  4.26125228e-02  1.45385070e-02 -2.61809099e-02
 -8.55612569e-03  3.41044478e-02  5.80871403e-02  2.10360885e-02
  9.20202676e-03  5.33556864e-02 -3.78961675e-02  7.10363919e-03
 -4.81327921e-02 -1.68317009e-03 -1.69199668e-02 -1.54205710e-02
  4.21466641e-02 -1.86046096e-03 -1.14875429e-05  3.93789858e-02
 -7.92814046e-03  4.13183197e-02 -3.82487513e-02  1.19009120e-02
  5.91590218e-02 -7.17932582e-02  3.91817885e-03 -3.09534799e-02
 -2.10253578e-02 -2.01134626e-02 -1.52507201e-02  1.25193074e-02
  2.95347571e-02  3.83191332e-02  1.45295821e-02  3.14703248e-02
 -1.17491484e-02 -1.88130457e-02  2.67630331e-02 -2.75423396e-02
  4.31421921e-02  6.73846807e-03 -2.47243177e-02 -1.85277127e-02
  7.30892364e-03 -4.28916467e-03 -4.82225642e-02 -5.07485718e-02
 -1.63041819e-02  2.46827658e-02 -1.86329603e-03 -2.11289600e-02
 -6.05235472e-02 -2.2906546

In [ ]:
# Quadrant quick start man pages: https://qdrant.tech/documentation/quick-start/
## create embaddings
import json
from qdrant_client.http.models import PointStruct
from qdrant_client import QdrantClient

instruction = "hello"

data = ["Hello AI world", "Bye AI world", "Buy me some pizza", "The lection is very good", "Nice to meet you"]

dataPoints = []
id = 0
for sentence in data:
    id += 1
    embeddings = model.encode([[instruction,sentence]])
    record = {"id": id, "emb": embeddings[0].tolist(), "payload": {"txt": sentence}}
    dataPoints.append(
        PointStruct(id=id, vector=embeddings[0].tolist(), payload=record)
    )

from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="hello",
    vectors_config=VectorParams(size=768, distance=Distance.DOT),
)


operation_info = client.upsert(
    collection_name="hello",
    wait=True,
    points=dataPoints,
)



In [ ]:
# Test questions
search_qq = ["how do you do"]
search_exp_id = 1
search_exp_hits = 0

for search_one_q in search_qq:
    search_embd = model.encode([[instruction,search_one_q]])
    search_result = client.search(
        collection_name="hello", query_vector=search_embd[0], limit=2
    )


    for res in search_result:
        print(str(res.id) + ": " + str(res.score) + ": " + str(res.payload['payload']))



5: 0.9094765186309814: {'txt': 'Nice to meet you'}
1: 0.8890740871429443: {'txt': 'Hello AI world'}


In [ ]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained('eugenesiow/bart-paraphrase')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
tokenizer = BartTokenizer.from_pretrained('eugenesiow/bart-paraphrase')



In [ ]:
input_sentence = "How do I update my user name or password?"

batch = tokenizer(input_sentence, return_tensors='pt')
generated_ids = model.generate(batch['input_ids'])
generated_sentence = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

print(generated_sentence)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['How do I reset my password?']
